## 大規模SAアニーリングTYTANチュートリアル

### シュミレイテッドアニーリング概要
シュミレイテッドアニーリング(SA)とは、組合せ最適化問題の解を求める手法です。  
組合せ最適化問題は、計算理論上NP困難と呼ばれる、組合せが多くなるにつれて計算量が指数関数的に増加してしまう問題があります。  
SAはこのNP困難の問題を解決するわけではなく、組み合わせ最適化問題の最適解(大域最適解)が得られる保証はありません、SAは近似解を求められる手法です。  

しかし、現実問題の多くは、問題の解き方そのものが分からないケースも多く、SAは実用する上では問題のない答えを得られる場合もあります。

### 組合せ最適化問題の解き方
SAで組み合わせ最適化問題を解く際には、コストと制約条件を定義します。  
コストは組合せ最適化問題の解を評価するための数式で、求まる解が良いか悪いかを定義できる数式です。

制約条件は、満たさないと解と認められない条件のことで、こちらも数式で定義します。


### コスト関数の例
0か1をとる変数$q_0,q_1,q_2$があるときに、
隣接する変数の値がすべて1となる変数の組み合わせのコスト関数Cは、隣接する変数の積にマイナスを掛けたもになります。
$$
C=-q_0q_1-q_0q_2-q_1q_2
$$
この例ははじめから、求める$q_0,q_1,q_2$の組み合わせは、$\{1,1,1\}$と分かっています。このときのコスト関数の値は、$C=-3$となり、$q_0,q_1,q_2$が$\{1,1,1\}$のときに最小値になることが分かります。  
このように、コスト関数は、求める状態が最小になるように定式化した式のことです。  
現実問題を解くときは当然、求める状態が分かっていないために、このコスト関数を定式化することで、SAを行うことでコスト関数が最小となるような状態を求めようと働きます。

### 制約条件の例
制約条件とは、たとえコスト関数が最小になったとしても、解とは認められない条件のことです。
コスト関数に制約条件の式を加えることで、制約条件を満たすような、状態を求めることができます。
コスト関数が、
$$
C=4q_0q_1-3q_1q_3+2q_2-q_3
$$  

で定式化されているときに、制約条件$D$が下記とします。
$$
D=(2q_0+q_1+q_2-2)^2
$$  
  

制約条件があるときには、コスト関数が最小になるように働きますが、制約条件が2乗で与えられているために、2乗のカッコの中が0になるようにSAが動作します。


コスト関数と制約条件を足した
$$
H=C+D
$$
がなるべく最小になるようにSAが動くようになります。

### QUBO式
上のコストと制約式を合わせた式$H$は、エネルギーもしくはハミルトニアンと呼ばれることもあります。  
SAのもととなるイジングモデルと呼ばれる物理モデルから由来しているためです。  
コスト関数と制約式は、どんなものでもSAで解けるわけではありません。  
コスト関数と制約式が2次形式と呼ばれる形に限られます。  
そのため、SAで解ける式のことを二次形式の制約なし二値変数最適化（Quadratic Unconstrained Binary Optimization）からQUBO式ともいいます。  
QUBO式は、一般に以下のようにあらわされます。

$$
H= \sum_{i,j} w_{i,j} q_i q_j
$$

また、$i$と$j$の要素に対する行列にもなっているため、QUBO行列と呼ばれることがあります。  
以上で説明した2次形式は、組合せ最適化問題の多くはこの形式に変換できることが知られています。

### 実際にTYTANで問題を解いてみる。
制約条件とコスト関数をもとめたら、実際にTYTANに解かせてみます。  
はじめに、TYTANモジュールをインストールします。

In [1]:
!pip install git+https://github.com/tytansdk/tytan.git

  Cloning https://github.com/tytansdk/tytan.git to c:\users\005087\appdata\local\temp\pip-req-build-buj8ox9_
  Resolved https://github.com/tytansdk/tytan.git to commit a7cd389f3bd7d76638822667983f77df4daa6a09
  Using cached httpcore-0.16.3-py3-none-any.whl (69 kB)
  Using cached httpx-0.23.3-py3-none-any.whl (71 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
  Using cached sympy-1.11.1-py3-none-any.whl (6.5 MB)
  Using cached ulid_py-1.1.0-py2.py3-none-any.whl (25 kB)
  Created wheel for tytan: filename=tytan-0.0.5-py3-none-any.whl size=14852 sha256=c78bc49fc1e60e366908753128e849c1f71373ee11ad6764da3f29ea43dbb14c
  Stored in directory: C:\Users\005087\AppData\Local\Temp\pip-ephem-wheel-cache-9e6088kl\wheels\82\f8\64\a3349ef2564f4525272b3140f2bfae4619c86ddb9caa137ecd
Successfully built tytan
  Attempting uninstall: mpmath
    Found existing installation: mpmath 1.2.1
    Uninstalling mpmath-1.2.1:
      Successfully uninstalled mpmath-1.2.1
  Attempting uninstall: sympy
    F

  Running command git clone -q https://github.com/tytansdk/tytan.git 'C:\Users\005087\AppData\Local\Temp\pip-req-build-buj8ox9_'


In [7]:
from tytan import *
import sympy as sym

# 変数を定義 q_0, q_1, q_2, q_3
q = sym.symbols("q_{0:4}")

#式を記述
#コスト関数
C= 4*q[0]*q[1]-3*q[1]*q[3]+2*q[2]-q[3]

#制約条件
D=(2*q[0]+q[1]+q[2]-2)**2

#式を結合
H = C+5*D

# Compileクラスを使用して、QUBOを取得
Q, offset = qubo.Compile(H).get_qubo()

API_KEY = "API key"

# サンプラーを選択
solver = sampler.NQSSampler()

# 計算
result = solver.run(Q, api_key=API_KEY)
print(result)

{'energy': -42.0, 'result': {'q_{0}': 0, 'q_{1}': 1, 'q_{2}': 1, 'q_{3}': 1}, 'time': 3.7242937088012695}


### 結果の確認
結果をみると、energyとありますが、これは、求まった状態に対するコスト関数と制約条件の式の数値が表示されています。  

状態は、$\{q_0,q_1,q_2,q_3  \}=\{0,1,1,1\}$です。
制約条件
$$
D=(2q_0+q_1+q_2-2)^2  
$$
$$
=(2*0+1+1-2)^2
$$

$$
=0
$$
と0になり制約が満たされていることが分かります。

このようにTYTANによるSAでは、解きたい問題のコストと制約を与えるだけで、その式に応じた結果が得られます。  
このあとは、現実問題で応用できそうな組み合わせ問題を解く例を見ていきます。